# 🎯 QEPC Player Props

**Quantum-Inspired Player Prop Predictions**

Features:
- 🎲 Monte Carlo simulation with quantum states
- 📊 Full distribution (floor to ceiling)
- 🛡️ Matchup-based adjustments
- 💰 Vegas line comparison
- ⭐ Edge detection

---

In [ ]:
# SETUP
import sys
from pathlib import Path

project_root = Path(r"C:\Users\wdors\qepc_project")
qepc_v2_path = project_root / "experimental" / "CLAUDE_REWRITE" / "qepc_v2"

if str(qepc_v2_path) not in sys.path:
    sys.path.insert(0, str(qepc_v2_path))

print(f"📁 Project root: {project_root}")

from qepc.sports.nba.props import PlayerPropsPredictor, quick_prop, MatchupAnalyzer

print("✅ Player Props module loaded!")

---
## 📥 Fetch Player Data (Run Once)

This fetches player stats from the NBA API. Run this once to populate your data, then skip on future runs.

In [ ]:
# OPTIONAL: Fetch fresh player data from NBA API
# Uncomment and run this cell to refresh player data

# !pip install nba_api  # Install if needed

import sys
sys.path.insert(0, str(qepc_v2_path / "scripts"))
from fetch_player_data import refresh_player_data

refresh_player_data(
      data_dir=project_root / "data",
      season="2024-25",
      full_logs=False,  # Set True for full game logs (slow)
  )

---
## 📊 Initialize Predictor

In [ ]:
# Initialize and load data
predictor = PlayerPropsPredictor(data_dir=project_root / "data")
predictor.prepare(verbose=True)

---
## 🏀 Single Player Prediction

In [ ]:
# Quick prediction for a single player
# Change these values!

PLAYER = "LeBron James"
PROP_TYPE = "pts"  # pts, reb, ast, fg3m, pra
VEGAS_LINE = 25.5
OPPONENT = "Golden State Warriors"  # Optional - for matchup adjustment

pred = predictor.predict_prop(
    player_name=PLAYER,
    prop_type=PROP_TYPE,
    vegas_line=VEGAS_LINE,
    opponent=OPPONENT,
    is_home=True,
)

if pred:
    print(f"\n🏀 {pred.player_name} vs {pred.opponent} - {PROP_TYPE.upper()}")
    print("=" * 50)
    print(f"""
📊 PREDICTION
   Expected: {pred.predicted_value:.1f} ± {pred.predicted_std:.1f}
   
📈 DISTRIBUTION
   Floor (P10): {pred.p10:.1f}
   Low (P25): {pred.p25:.1f}
   Median: {pred.p50:.1f}
   High (P75): {pred.p75:.1f}
   Ceiling (P90): {pred.p90:.1f}

🛡️ ADJUSTMENTS APPLIED
   Matchup: {pred.matchup_adj:+.1f}%
   Pace: {pred.pace_adj:+.1f}%
   Home: {pred.home_adj:+.1f}%
   Form: {pred.form_adj:+.1f}%

💰 VS VEGAS (Line: {pred.vegas_line})
   Over: {pred.over_prob:.1%}
   Under: {pred.under_prob:.1%}
   Confidence: {pred.confidence:.1%}
    """)
    
    if pred.has_edge:
        print(f"   ⭐ EDGE: {pred.edge} ({pred.edge_size:.1%})")

---
## 🔍 Search for Players

In [ ]:
# Search for players by name
SEARCH = "curry"

matches = predictor.search_players(SEARCH)
print(f"🔍 Players matching '{SEARCH}':")
for name in matches:
    print(f"   {name}")

---
## 🛡️ Matchup Analysis

In [ ]:
# View best and worst matchups for a stat category
STAT = "pts"  # pts, reb, ast, fg3m

analyzer = predictor._matchup_analyzer

print(f"\n🎯 BEST MATCHUPS FOR {STAT.upper()} (Teams that allow most)")
print("=" * 50)
best = analyzer.get_best_matchups(STAT, n=10)
if not best.empty:
    for _, row in best.iterrows():
        print(f"   {row['Team']:25} Factor: {row['Factor']:.2f}")

print(f"\n🛡️ WORST MATCHUPS FOR {STAT.upper()} (Teams that allow least)")
print("=" * 50)
worst = analyzer.get_worst_matchups(STAT, n=10)
if not worst.empty:
    for _, row in worst.iterrows():
        print(f"   {row['Team']:25} Factor: {row['Factor']:.2f}")

---
## 📋 Predict a Full Slate

In [ ]:
# Define your slate: (player_name, prop_type, vegas_line)
SLATE = [
    ("LeBron James", "pts", 25.5),
    ("Stephen Curry", "pts", 26.5),
    ("Stephen Curry", "fg3m", 4.5),
    ("Nikola Jokic", "ast", 8.5),
    ("Nikola Jokic", "reb", 12.5),
    ("Anthony Edwards", "pts", 24.5),
    ("Luka Doncic", "pra", 48.5),
    ("Jayson Tatum", "pts", 27.5),
    ("Giannis Antetokounmpo", "reb", 11.5),
    ("Trae Young", "ast", 10.5),
]

print("\n🏀 SLATE PREDICTIONS")
print("=" * 60)

predictions = predictor.predict_slate(SLATE, verbose=True)

In [ ]:
# Show as table
if predictions:
    df = predictor.predictions_to_dataframe(predictions)
    display(df)

---
## ⭐ Find Best Edges

In [ ]:
# Find props with edges (>55% probability)
MIN_EDGE = 0.55  # Minimum probability to consider an edge

edges = predictor.find_edges(predictions, min_edge=MIN_EDGE)

if edges:
    print(f"\n⭐ FOUND {len(edges)} EDGES (>{MIN_EDGE:.0%})")
    print("=" * 60)
    
    for pred in edges:
        print(f"\n{pred.player_name} {pred.prop_type.upper()}")
        print(f"   Prediction: {pred.predicted_value:.1f}")
        print(f"   Vegas Line: {pred.vegas_line}")
        print(f"   Edge: {pred.edge} ({pred.edge_size:.1%})")
        print(f"   Confidence: {pred.confidence:.0%}")
else:
    print(f"\n❌ No edges found above {MIN_EDGE:.0%}")

---
## 📊 Compare Multiple Props for One Player

In [ ]:
# Analyze all props for a single player
PLAYER = "Nikola Jokic"
OPPONENT = "Los Angeles Lakers"

# Define props and lines
player_props = {
    'pts': 25.5,
    'reb': 12.5,
    'ast': 8.5,
    'pra': 48.5,
    'fg3m': 0.5,
}

print(f"\n🏀 {PLAYER} vs {OPPONENT} - All Props")
print("=" * 60)

for prop, line in player_props.items():
    pred = predictor.predict_prop(PLAYER, prop, vegas_line=line, opponent=OPPONENT)
    if pred:
        edge_str = f"⭐{pred.edge}" if pred.has_edge else ""
        print(f"{prop.upper():5} | Pred: {pred.predicted_value:5.1f} | Line: {line:5.1f} | "
              f"Over: {pred.over_prob:4.0%} | Under: {pred.under_prob:4.0%} | {edge_str}")

---
## 🎲 Understanding Quantum Prop Modeling

### Performance States (Superposition)
Each player exists in a superposition of 5 performance states:

| State | Multiplier | Base Probability | Description |
|-------|------------|-----------------|-------------|
| CEILING | 1.35x | 5% | Everything clicks |
| ELEVATED | 1.15x | 20% | Above average night |
| BASELINE | 1.00x | 50% | Typical performance |
| FLOOR | 0.85x | 20% | Off night |
| BUST | 0.50x | 5% | Disaster game |

### Probability Shifts
Context shifts these probabilities:
- **Hot streak** → More ELEVATED/CEILING
- **Bad matchup** → More FLOOR/BUST
- **Home game** → Slight boost
- **High variance player** → More extreme states

### Matchup Effects (Interference)
- **Constructive interference**: Bad defense → boosts stats
- **Destructive interference**: Elite defense → reduces stats

### Edge Detection
- **Over 55%** = Potential edge (accounts for ~-110 juice)
- **Over 60%** = Strong edge
- **Over 65%** = Very strong edge

---
⚠️ **Disclaimer**: For entertainment/research only. Gamble responsibly.